In [11]:
!pip3 install pandas dash

In [12]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-07-04 17:58:02--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-07-04 17:58:02 (32.0 MB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]



In [13]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2022-07-04 17:58:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2.1K) [text/x-python]
Saving to: ‘spacex_dash_app.py.1’

spacex_dash_app.py. 100%[===================>]   2.06K  --.-KB/s    in 0s      

2022-07-04 17:58:04 (29.3 MB/s) - ‘spacex_dash_app.py.1’ saved [2110/2110]



In [14]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [15]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [16]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options = [
                                            {'label':'All sites', 'value' : 'ALL'},
                                            {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                            {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                            {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                            {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                        ], value = 'ALL', placeholder='Select Launch Site here',
                                            searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min = 0, 
                                                max = 10000,step = 1000,
                                                value = [min_payload,max_payload]
                                           ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ]
                )

In [17]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id= 'success-pie-chart', component_property='figure'),
                Input(component_id ='site-dropdown', component_property='value')
                )
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
    if entered_site == 'ALL':
        fig_pie = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Site-wise Success count(%)')
        fig_pie.update_layout(title_x=0.45,
                            font=dict(
                                size=20,
                                color="RebeccaPurple"))
        return fig_pie
    else:
        fig_pie = px.pie(filtered_df, 
        names= 'class', 
        title = 'Launch Success rate of site ' + entered_site)
        fig_pie.update_layout(title_x=0.45,legend_title_text = "(1-Success, 0-Fail)",
                            font=dict(size=20,
                                color="RebeccaPurple"))
        return fig_pie
        
        # return the outcomes piechart for a selected site

In [18]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id= 'success-payload-scatter-chart', component_property='figure'),
                    [Input(component_id ='site-dropdown', component_property='value'),
                    Input(component_id='payload-slider', component_property='value')
                    ]
)
def get_scatter_chart(entered_site,payload):
    print(payload)
    low, high = payload
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if entered_site == 'ALL':
        fig_scat = px.scatter(filtered_df, x= 'Payload Mass (kg)',
                        y= 'class',
                        color = 'Booster Version Category',
                        title = 'Correlation : payload Vs class for all sites',
                        )
        fig_scat.update_layout(title_x=0.45,
                            font=dict(
                                size=20,
                                color="RebeccaPurple"))
        return fig_scat
    else:
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig_scat= px.scatter(filtered_df, x= 'Payload Mass (kg)',
                        y= 'class',
                        title = "Correlation : payload Vs class for "+entered_site,
                        color = "Booster Version Category")
        fig_scat.update_layout(title_x=0.45,
                            font=dict(
                                size=20,
                                color="RebeccaPurple"))
        return fig_scat


In [ ]:
# Run the app

if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
